In [136]:
from gc_utils import *

alg = Algebra(4)
ga2 = Algebra(2)
locals().update(alg.blades)

In [137]:
# 2d general linear transform
def random_linear_trans(alg: Algebra):
    matrix = np.random.rand(alg.d, alg.d)
    return matrix2trans(matrix, alg)

def random_ortho_trans(alg: Algebra):
    from scipy.stats import ortho_group
    matrix = ortho_group.rvs(dim=alg.d)
    return matrix2trans(matrix, alg)

f = random_linear_trans(ga2)
x = random_r_blade(1, ga2)
f(x)

0.0843 𝐞₁ + 0.392 𝐞₂

In [138]:
# 2d polar decomposition
# f = SUU
df = derivative(f, x, ga2, grade=1)
UU = df / norm(df)
S = lambda x: UU * f(x)
S(x)*UU, f(x)

(0.0843 𝐞₁ + 0.392 𝐞₂, 0.0843 𝐞₁ + 0.392 𝐞₂)

In [139]:
# UU is rotor, S is symmetric
UU, curl(S, x, ga2, grade=1)

(0.999 + 0.0357 𝐞₁₂, 2.78e-11 𝐞₁₂)

In [140]:
#6.3 remove the condition of 2 dimension
a, b, x = random_r_vectors(3, alg)
g = lambda x: x + (x|a)*b
abframe = [a,b]
r_frame = reciprocal(abframe)
dg = 2 + a*b
derivative(g, x, alg, grade=1, frame=abframe, r_frame=r_frame), dg

(2.87 + -0.115 𝐞₁₂ + 0.527 𝐞₁₃ + 0.339 𝐞₁₄ + 0.597 𝐞₂₃ + 0.346 𝐞₂₄ + 0.181 𝐞₃₄ + 1.8e-16 𝐞₁₂₃₄,
 2.87 + -0.115 𝐞₁₂ + 0.527 𝐞₁₃ + 0.339 𝐞₁₄ + 0.597 𝐞₂₃ + 0.346 𝐞₂₄ + 0.181 𝐞₃₄)

In [141]:
#6.3
dg_norm = norm(dg)
UU = dg / dg_norm
U = simple_rotor_sqrt(UU)
S = lambda x: U.sw(g(x))
curl(S, x, alg, grade=1), U.reverse().sw(S(x)), g(x)

(-1.11e-10 𝐞₁₂ + 1.11e-10 𝐞₁₃ + -1.11e-10 𝐞₁₄ + -4.44e-10 𝐞₂₃ + -3.33e-10 𝐞₂₄ + -1.11e-10 𝐞₃₄,
 1.02 𝐞₁ + 0.76 𝐞₂ + 2.27 𝐞₃ + 1.75 𝐞₄,
 1.02 𝐞₁ + 0.76 𝐞₂ + 2.27 𝐞₃ + 1.75 𝐞₄)

In [142]:
#6.6
frame = alg.frame
i, j = 1, 3
aij = (1/np.sqrt(2))*(frame[i] + frame[j])
[aij.sw(x) for x in frame]

[-1.0 𝐞₁, 1.0 𝐞₄, -1.0 𝐞₃, 1.0 𝐞₂]

In [143]:
#6.7
alpha = 3
Sij = lambda x: x + (alpha * (x|frame[i]) * frame[j])
[Sij(x) for x in frame]

[1 𝐞₁, 1 𝐞₂ + 3 𝐞₄, 1 𝐞₃, 1 𝐞₄]

In [144]:
#7.2
g = random_linear_trans(alg)
g_ = lambda x, y: x | g(y)
derivative(lambda x: g_(x, b), x, alg, grade=1), g(b)

(0.512 𝐞₁ + 0.978 𝐞₂ + 0.0723 𝐞₃ + 1.06 𝐞₄,
 0.512 𝐞₁ + 0.978 𝐞₂ + 0.0723 𝐞₃ + 1.06 𝐞₄)

In [145]:
# check nonsingular
outermorphism(g, alg.pseudoscalar((1,)), alg)

0.0242 𝐞₁₂₃₄

In [146]:
#7.5 orthogonal transformation is its isometry
f = random_ortho_trans(alg)
x | f(b), f(x) | f(f(b))

(0.0582, 0.0582)

In [147]:
# skew bilinear form
gm = skew_part(g, alg)
G = simplicial_derivative(lambda vectors: vectors[0] | gm(vectors[1]), np.zeros(2), alg)
x | gm(b), x | (b | G)

(0.36, 0.36)

In [148]:
#7.6b f is not an isometry of gm
adjoint_outermorphism(f, G, alg), G

(-0.213 𝐞₁₂ + -0.00947 𝐞₁₃ + -0.212 𝐞₁₄ + -0.237 𝐞₂₃ + 0.308 𝐞₂₄ + 0.0974 𝐞₃₄,
 -0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄)

In [149]:
#7.8 the inverse of gm
blades = bivector_split(G, alg)
gm(sum(-x | blade/normsq(blade) for blade in blades)), x

(0.711 𝐞₁ + 0.748 𝐞₂ + 0.734 𝐞₃ + 0.864 𝐞₄,
 0.711 𝐞₁ + 0.748 𝐞₂ + 0.734 𝐞₃ + 0.864 𝐞₄)

In [150]:
#7.12 assume splitted blades are normal
blades_norm = [norm(b) for b in blades]
blades = [b / n for b, n in zip(blades, blades_norm)]
gprod(blades)

5.55e-16 + -8.67e-18 𝐞₁₂ + 4.68e-17 𝐞₁₃ + -1.73e-18 𝐞₁₄ + -6.94e-18 𝐞₂₃ + -1.39e-17 𝐞₃₄ + -1.0 𝐞₁₂₃₄

In [151]:
#7.17
theta = np.random.rand(2)
rotors = [blade_exp(t*blade) for t, blade in zip(theta, blades)]
[(U.sw(G), G) for U in rotors for G in blades]

[(-0.676 𝐞₁₂ + -0.337 𝐞₁₃ + 0.632 𝐞₁₄ + 0.048 𝐞₂₃ + 0.127 𝐞₂₄ + 0.108 𝐞₃₄,
  -0.676 𝐞₁₂ + -0.337 𝐞₁₃ + 0.048 𝐞₂₃ + 0.632 𝐞₁₄ + 0.127 𝐞₂₄ + 0.108 𝐞₃₄),
 (-0.108 𝐞₁₂ + 0.127 𝐞₁₃ + -0.048 𝐞₁₄ + -0.632 𝐞₂₃ + -0.337 𝐞₂₄ + 0.676 𝐞₃₄,
  -0.108 𝐞₁₂ + 0.127 𝐞₁₃ + -0.632 𝐞₂₃ + -0.048 𝐞₁₄ + -0.337 𝐞₂₄ + 0.676 𝐞₃₄),
 (-0.676 𝐞₁₂ + -0.337 𝐞₁₃ + 0.632 𝐞₁₄ + 0.048 𝐞₂₃ + 0.127 𝐞₂₄ + 0.108 𝐞₃₄,
  -0.676 𝐞₁₂ + -0.337 𝐞₁₃ + 0.048 𝐞₂₃ + 0.632 𝐞₁₄ + 0.127 𝐞₂₄ + 0.108 𝐞₃₄),
 (-0.108 𝐞₁₂ + 0.127 𝐞₁₃ + -0.048 𝐞₁₄ + -0.632 𝐞₂₃ + -0.337 𝐞₂₄ + 0.676 𝐞₃₄,
  -0.108 𝐞₁₂ + 0.127 𝐞₁₃ + -0.632 𝐞₂₃ + -0.048 𝐞₁₄ + -0.337 𝐞₂₄ + 0.676 𝐞₃₄)]

In [152]:
#7.18
first_vecs = [blade_split(blade, alg)[0] for blade in blades]
first_vecs = [v/norm(v) for v in first_vecs]
second_vecs = [v | blade for v, blade in zip(first_vecs, blades)]
inter_plane_rotors = [simple_rotor_sqrt(a * b) for a, b in (first_vecs, second_vecs)]
U = gprod(inter_plane_rotors)
terms_ratio(U.sw(blades[0]), blades[1])

array([1., 1., 1., 1., 1., 1.])

In [153]:
G_normalized = sum(blades)
G_normalized, U.sw(G_normalized)

(-0.784 𝐞₁₂ + -0.21 𝐞₁₃ + 0.584 𝐞₁₄ + -0.584 𝐞₂₃ + -0.21 𝐞₂₄ + 0.784 𝐞₃₄,
 -0.784 𝐞₁₂ + -0.21 𝐞₁₃ + 0.584 𝐞₁₄ + -0.584 𝐞₂₃ + -0.21 𝐞₂₄ + 0.784 𝐞₃₄)

In [154]:
# we can recover this "normalization"
sum(P(G_normalized, b) * n for b, n in zip(blades, blades_norm)), G

(-0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄,
 -0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄)

In [155]:
#7.21 zero if not onto the decomposed blades 
sympletic_basis = first_vecs + second_vecs
list(zip((v | (w | G_normalized) for v, w in product(sympletic_basis, repeat=2)), (product(['f', 'fp', 's', 'sp'], repeat=2))))

[(2.78e-17, ('f', 'f')),
 (7.29e-17, ('f', 'fp')),
 (-1.0, ('f', 's')),
 (, ('f', 'sp')),
 (-5.55e-17, ('fp', 'f')),
 (-3.45e-17, ('fp', 'fp')),
 (-5.55e-17, ('fp', 's')),
 (-1.0, ('fp', 'sp')),
 (1.0, ('s', 'f')),
 (5.04e-17, ('s', 'fp')),
 (-1.39e-17, ('s', 's')),
 (2.78e-17, ('s', 'sp')),
 (1.85e-18, ('sp', 'f')),
 (1.0, ('sp', 'fp')),
 (-1.74e-17, ('sp', 's')),
 (3.45e-17, ('sp', 'sp'))]

In [156]:
# Recover the normalization
[gm(v)/n for v, n in zip(sympletic_basis, blades_norm*2)], sympletic_basis

([0.981 𝐞₁ + 0.0696 𝐞₃ + 0.183 𝐞₄,
  -0.0635 𝐞₁ + -0.445 𝐞₂ + 0.893 𝐞₃ + -1.12e-16 𝐞₄,
  -0.0925 𝐞₁ + -0.69 𝐞₂ + -0.35 𝐞₃ + 0.627 𝐞₄,
  -0.161 𝐞₁ + 0.571 𝐞₂ + 0.273 𝐞₃ + 0.757 𝐞₄],
 [0.0925 𝐞₁ + 0.69 𝐞₂ + 0.35 𝐞₃ + -0.627 𝐞₄,
  0.161 𝐞₁ + -0.571 𝐞₂ + -0.273 𝐞₃ + -0.757 𝐞₄,
  0.981 𝐞₁ + 1.39e-17 𝐞₂ + 0.0696 𝐞₃ + 0.183 𝐞₄,
  -0.0635 𝐞₁ + -0.445 𝐞₂ + 0.893 𝐞₃ + -2.78e-17 𝐞₄])

In [157]:
#7.26a symplectic transformations of gm (isometries of skew bilinear form)
# conditions: unity and sympletic basis
translation = lambda x: (
    x + 
    2*(x|second_vecs[1]) * first_vecs[1] + 
    3*(x|second_vecs[0]) * first_vecs[0]
    )

outermorphism(translation, G_normalized, alg), G_normalized, outermorphism(translation, G, alg), G

(-0.784 𝐞₁₂ + -0.21 𝐞₁₃ + 0.584 𝐞₁₄ + -0.584 𝐞₂₃ + -0.21 𝐞₂₄ + 0.784 𝐞₃₄,
 -0.784 𝐞₁₂ + -0.21 𝐞₁₃ + 0.584 𝐞₁₄ + -0.584 𝐞₂₃ + -0.21 𝐞₂₄ + 0.784 𝐞₃₄,
 -0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄,
 -0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄)

In [158]:
#7.26b
rotation = lambda x: (
    x + 
    4*((x|first_vecs[1]) * second_vecs[1] + (x|second_vecs[0]) * first_vecs[0])
    )

outermorphism(rotation, G_normalized, alg), G_normalized, outermorphism(rotation, G, alg), G

(-0.784 𝐞₁₂ + -0.21 𝐞₁₃ + 0.584 𝐞₁₄ + -0.584 𝐞₂₃ + -0.21 𝐞₂₄ + 0.784 𝐞₃₄,
 -0.784 𝐞₁₂ + -0.21 𝐞₁₃ + 0.584 𝐞₁₄ + -0.584 𝐞₂₃ + -0.21 𝐞₂₄ + 0.784 𝐞₃₄,
 -0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄,
 -0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄)

In [159]:
#7.26c
outermorphism(lambda x: rotation(translation(x)), G, alg), G

(-0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄,
 -0.0977 𝐞₁₂ + 0.0408 𝐞₁₃ + 0.0174 𝐞₁₄ + -0.311 𝐞₂₃ + -0.159 𝐞₂₄ + 0.343 𝐞₃₄)

In [160]:
#7.28
gp = sym_part(g, alg)
matrix = trans2matrix(gp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
neg_vecs, pos_vecs, null_vecs = [], [], []
for e, v in zip(eigvalues, eigvecs):
    if e < -1e-6:
        neg_vecs.append(v)
    elif e > 1e-6:
        pos_vecs.append(v)
    else:
        null_vecs.append(v)

neg_blade = wedge(neg_vecs)
nv_sign = (-1) ** len(neg_vecs) 
c = lambda x: nv_sign*neg_blade.sw(x)
h = lambda x: sum(np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))

x | gp(b), h(x)|c(h(b)), c(x)|h(h(b))

(1.71, 1.71, 1.71)

In [161]:
#7.28 If the symmetric transformation is positive definite (Euclidean metric)
gpp = lambda x: sum(abs(e) * P(x, v) for e, v in zip(eigvalues, eigvecs))
hinv = lambda x: sum(1/np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))
r = 2 # a arbitrary integer <= n
Q = random_r_blade(r, alg)
Q = Q / norm(Q)
f = lambda x: -hinv(Q.sw(h(x)))
x | gpp(b), f(x) | gpp(f(b)), x, f(x)

(1.75,
 1.75,
 0.711 𝐞₁ + 0.748 𝐞₂ + 0.734 𝐞₃ + 0.864 𝐞₄,
 0.712 𝐞₁ + -2.25 𝐞₂ + 2.66 𝐞₃ + 1.79 𝐞₄)

In [162]:
#7.33 isometry of c
def ortho_split(x, A):
    xp = P(x, A)
    return xp, x-xp

xp, xo = ortho_split(x, neg_blade)
bp, bo = ortho_split(b, neg_blade)
x | c(b), (xo | bo) - (xp | bp)


(0.712, 0.712)

In [163]:
# Let R be a rotor (see next section) preserves the reflected subspace
R = 1
if len(neg_vecs) >= 2:
    R *= blade_exp(3.3 * wedge(neg_vecs[:2]))
if len(pos_vecs) >= 2:
    R *= blade_exp(5.5 * wedge(pos_vecs[:2]))
R.sw(neg_blade), neg_blade

(0.289 𝐞₁₂ + -0.671 𝐞₁₃ + 0.0576 𝐞₁₄ + -0.0215 𝐞₂₃ + 0.271 𝐞₂₄ + -0.624 𝐞₃₄,
 0.289 𝐞₁₂ + -0.671 𝐞₁₃ + 0.0576 𝐞₁₄ + -0.0215 𝐞₂₃ + 0.271 𝐞₂₄ + -0.624 𝐞₃₄)

In [164]:
# an isometry of a pseudoEuclidean metric
f = lambda x: -hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), x, f(x)

(1.71,
 1.71,
 0.711 𝐞₁ + 0.748 𝐞₂ + 0.734 𝐞₃ + 0.864 𝐞₄,
 -1.37 𝐞₁ + 2.71 𝐞₂ + 0.596 𝐞₃ + -1.57 𝐞₄)

So a rotation preserves the reflected subspace is an isometry, any other isometry? The next section encode the metric into the algebra. So an orthogonal transform becomes an isometry of the vector space.

In [165]:
# signature
[v | c(v) for v in eigvecs]

[-1.0, -1.0, 1.0, 1.0]

In [166]:
p, q, r = 0,0,0
for e in eigvalues:
    if e > 1e-6:
        p += 1
    elif e < -1e-6:
        q += 1
    else:
        r += 1
p, q, r

(2, 2, 0)

In [167]:
#8.2 Say the reflected subspace is 1d
alg = Algebra(3, 1)
locals().update(alg.blades)
x, y, u = random_r_vectors(3, alg)
u = u / norm(u)
R = lambda x: - u.sw(x)
R(x) | R(y), x | y

(0.214, 0.214)

Cartan-Dieudonne theorem: An isometry is a composition of at most d reflections. Identifying invariants of isometries as elements of geometry leads to modern developments.

In [168]:
#8.3 Cartan-Dieudonne theorem: An isometry is a composition of at most d reflections.

k = 3 # k <= d
vectors = random_r_vectors(k, alg)
p = sum((v**2)[0] > 0 for v in vectors)
U = gprod(vectors) # a versor, of either even or odd grades
f = lambda x: (-1) ** k * U * x / U
f_ = lambda x: (-1) ** p * U.sw(x) / normsq(U) #8.8b
f(x) | f(y), x | y, f_(x) | f_(y)

(0.214 + 5.43e-17 𝐞₁₂ + -4.55e-17 𝐞₁₃ + 5.24e-18 𝐞₁₄ + -3.58e-17 𝐞₂₃ + 2.73e-17 𝐞₂₄ + -3.12e-17 𝐞₃₄,
 0.214,
 0.214)

In [169]:
#8.8
def random_versor(r, alg):
    return gprod(random_r_vectors(r, alg))

def random_versor_(r, alg):
    vectors = random_r_vectors(r, alg)
    p = sum((v**2)[0] > 0 for v in vectors)
    return gprod(vectors), p
k = 3
for _ in range(10):
    U, p = random_versor_(k, alg)
    U = U/norm(U)
    print(k - p, U.reverse() * U)

2 1.0 + -2.84e-14 𝐞₁₃ + -2.84e-14 𝐞₁₄ + -7.11e-15 𝐞₂₃ + -6.39e-14 𝐞₃₄ + 8.53e-14 𝐞₁₂₃₄
0 1.0 + 1.25e-16 𝐞₁₄ + 4.16e-17 𝐞₂₃ + 2.22e-16 𝐞₂₄ + -3.33e-16 𝐞₁₂₃₄
1 -1.0 + -2.22e-16 𝐞₁₂₃₄
0 1.0 + -1.67e-15 𝐞₁₃ + -1.14e-15 𝐞₁₄ + -3.39e-15 𝐞₃₄ + 2.66e-15 𝐞₁₂₃₄
0 1.0 + 3.99e-17 𝐞₂₄ + -3.47e-18 𝐞₁₂₃₄
0 1.0 + 1.47e-17 𝐞₁₃ + 3.56e-17 𝐞₁₄ + -1.39e-17 𝐞₂₃ + -9.71e-17 𝐞₂₄ + 4.16e-17 𝐞₃₄ + -1.04e-17 𝐞₁₂₃₄
0 1.0 + 2.78e-17 𝐞₁₃ + 6.94e-17 𝐞₁₄ + -5.55e-17 𝐞₂₃ + 5.55e-17 𝐞₂₄ + 9.71e-17 𝐞₃₄ + -3.33e-16 𝐞₁₂₃₄
1 -1.0 + 2.78e-17 𝐞₁₃ + 6.94e-17 𝐞₂₄ + 1.39e-17 𝐞₃₄ + 4.16e-17 𝐞₁₂₃₄
0 1.0 + 2.78e-17 𝐞₁₂ + 6.94e-18 𝐞₁₃ + -5.55e-17 𝐞₂₄ + -2.78e-17 𝐞₃₄ + -1.73e-18 𝐞₁₂₃₄
0 1.0 + 1.04e-17 𝐞₁₂ + 2.26e-17 𝐞₁₃ + -3.47e-18 𝐞₁₄ + -4.08e-17 𝐞₂₃ + 1.39e-17 𝐞₂₄ + 4.86e-17 𝐞₁₂₃₄


In [170]:
# versors (Pin) are 2:1 homomorphisc to orthogonal transformations (O)
U = random_versor(3, alg)
U.sw(x), (-U).sw(x)

(-0.517 𝐞₁ + -0.0489 𝐞₂ + 0.167 𝐞₃ + 0.486 𝐞₄,
 -0.517 𝐞₁ + -0.0489 𝐞₂ + 0.167 𝐞₃ + 0.486 𝐞₄)

In [171]:
#8.5
I = alg.pseudoscalar((1,))
I.sw(x), x

(0.347 𝐞₁ + 0.12 𝐞₂ + 0.741 𝐞₃ + 0.106 𝐞₄,
 0.347 𝐞₁ + 0.12 𝐞₂ + 0.741 𝐞₃ + 0.106 𝐞₄)

In [172]:
#8.7 orthogonal transformations (O) with determinant 1 (SO) correspond to even versors (Spin)
for k in range(1,5):
    U = gprod(random_r_vectors(k, alg)) # a versor, of either even or odd grades
    f = lambda x: (-1) ** (k*alg.d) * U.sw(x)/normsq(U)
    print(k, outermorphism(f, I, alg))

1 -1.0 𝐞₁₂₃₄
2 1.0 𝐞₁₂₃₄
3 -1.0 𝐞₁₂₃₄
4 1.0 𝐞₁₂₃₄


In [173]:
#8.9 rotors (Spin+) are a part of even versors (Spin), correpond to a part of SO as rotations (SO+)
def random_even_versor(alg):
    d = alg.d
    if (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


def random_rotor(alg, max_iter=100):
    for _ in range(max_iter):
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)
        

def random_rotor_count(alg, max_iter=100):
    count = 0
    for _ in range(max_iter):
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            count += 1
    return count

[(p, q, r, random_rotor_count(Algebra(p, q, r), 2000)/2000) for p, q, r in ((3,1,0),(1,3,0),(4,1,0),(3,0,1),(2,2,0))]

[(3, 1, 0, 0.654),
 (1, 3, 0, 0.6605),
 (4, 1, 0, 0.806),
 (3, 0, 1, 1.0),
 (2, 2, 0, 0.499)]

versors (Pin) are products of {1, a, b, ab} and a rotor

In [174]:
#8.12
def random_simple_rotor(alg):
    while True:
        S = random_versor(2, alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)

In [175]:
#8.13
S = random_simple_rotor(alg)
u = x | S.grade(2)
u = u/norm(u)
v = S.sw(u)
v, S ** 2 * u, v ** 2, u ** 2, v * u, S ** 2

(0.888 𝐞₁ + 0.988 𝐞₂ + -0.289 𝐞₃ + 0.921 𝐞₄,
 0.888 𝐞₁ + 0.988 𝐞₂ + -0.289 𝐞₃ + 0.921 𝐞₄ + 3.59e-16 𝐞₁₂₃ + -1.13e-16 𝐞₁₂₄ + -3.66e-16 𝐞₁₃₄ + 3.76e-16 𝐞₂₃₄,
 1.0,
 1.0,
 -0.993 + 0.0425 𝐞₁₂ + 0.131 𝐞₁₃ + 0.0655 𝐞₁₄ + 0.159 𝐞₂₃ + 0.0288 𝐞₂₄ + -0.157 𝐞₃₄,
 -0.993 + 0.0425 𝐞₁₂ + 0.131 𝐞₁₃ + 0.0655 𝐞₁₄ + 0.159 𝐞₂₃ + 0.0288 𝐞₂₄ + -0.157 𝐞₃₄ + 3.89e-16 𝐞₁₂₃₄)

In [176]:
S ** 2 * (S ** 2).reverse(), S * S.reverse()

(1.0 + 6.51e-19 𝐞₁₂ + 5.99e-18 𝐞₁₃ + 6.36e-18 𝐞₁₄ + 2.3e-18 𝐞₂₃ + -5.67e-18 𝐞₂₄ + -1.12e-17 𝐞₃₄ + -7.81e-16 𝐞₁₂₃₄,
 1.0 + 2.78e-17 𝐞₁₂ + 2.78e-17 𝐞₃₄ + -3.89e-16 𝐞₁₂₃₄)

In [177]:
#8.15
(S ** 2).reverse(), u * v, v * u

(-0.993 + -0.0425 𝐞₁₂ + -0.131 𝐞₁₃ + -0.0655 𝐞₁₄ + -0.159 𝐞₂₃ + -0.0288 𝐞₂₄ + 0.157 𝐞₃₄ + 3.89e-16 𝐞₁₂₃₄,
 -0.993 + -0.0425 𝐞₁₂ + -0.131 𝐞₁₃ + -0.0655 𝐞₁₄ + -0.159 𝐞₂₃ + -0.0288 𝐞₂₄ + 0.157 𝐞₃₄,
 -0.993 + 0.0425 𝐞₁₂ + 0.131 𝐞₁₃ + 0.0655 𝐞₁₄ + 0.159 𝐞₂₃ + 0.0288 𝐞₂₄ + -0.157 𝐞₃₄)

In [178]:
#8.16
S2 = S ** 2
blade = S2.grade(2)
bladesq = (blade ** 2)
bladesq

-0.0145 + -6.51e-19 𝐞₁₂ + 1.04e-17 𝐞₁₂₃₄

In [179]:
# 3 types of rotors (Spin+)
def type_convert(n):
    if n > 1e-6:
        return 1
    elif n < -1e-6:
        return -1
    else:
        return 0

# Unfortunately parabolic rotors are of measure zero, 
# so this function has zero chance generating them
def random_simple_rotor(alg, signature=None, max_iter=100):
    for _ in range(max_iter):
        S = random_versor(2, alg)
        if signature is not None:
            if type_convert((S.grade(2) ** 2)[0]) != signature:
                continue
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)
        
[random_simple_rotor(alg, signature=i) for i in range(-1, 2)]

[0.877 + 0.408 𝐞₁₂ + 1.6 𝐞₁₃ + 1.57 𝐞₁₄ + -0.353 𝐞₂₃ + -0.281 𝐞₂₄ + 0.26 𝐞₃₄,
 None,
 1.26 + 0.137 𝐞₁₂ + 0.302 𝐞₁₃ + -0.289 𝐞₁₄ + 0.26 𝐞₂₃ + -0.536 𝐞₂₄ + -0.634 𝐞₃₄]

In [180]:
def simple_rotor_log(R: MultiVector, tol=1e-6):
    blade = R.grade(2)
    signature = (blade ** 2)[0]
    if signature > tol:
        return np.arccosh(R[0]) / norm(blade) * blade
    if signature < -tol:
        return np.arccos(R[0]) / norm(blade) * blade
    else:
        return blade

In [181]:
#8.17a elliptic rotor (rotation), run several times you can see sqrt allows a sign change
S = random_simple_rotor(alg, signature=-1)
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), blade_exp(b - 6*np.pi*b/norm(b)), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(-0.737 + -0.693 𝐞₁₂ + 0.163 𝐞₁₃ + -0.137 𝐞₁₄ + 0.59 𝐞₂₃ + 0.563 𝐞₂₄ + -0.249 𝐞₃₄ + -5.55e-17 𝐞₁₂₃₄,
 -0.737 + -0.693 𝐞₁₂ + 0.163 𝐞₁₃ + -0.137 𝐞₁₄ + 0.59 𝐞₂₃ + 0.563 𝐞₂₄ + -0.249 𝐞₃₄,
 -0.737 + -0.693 𝐞₁₂ + 0.163 𝐞₁₃ + -0.137 𝐞₁₄ + 0.59 𝐞₂₃ + 0.563 𝐞₂₄ + -0.249 𝐞₃₄,
 0.363 + -0.955 𝐞₁₂ + 0.225 𝐞₁₃ + -0.189 𝐞₁₄ + 0.813 𝐞₂₃ + 0.776 𝐞₂₄ + -0.344 𝐞₃₄,
 0.363 + -0.955 𝐞₁₂ + 0.225 𝐞₁₃ + -0.189 𝐞₁₄ + 0.813 𝐞₂₃ + 0.776 𝐞₂₄ + -0.344 𝐞₃₄,
 0.363 + -0.955 𝐞₁₂ + 0.225 𝐞₁₃ + -0.189 𝐞₁₄ + 0.813 𝐞₂₃ + 0.776 𝐞₂₄ + -0.344 𝐞₃₄ + -7.65e-17 𝐞₁₂₃₄,
 0.363 + -0.955 𝐞₁₂ + 0.225 𝐞₁₃ + -0.189 𝐞₁₄ + 0.813 𝐞₂₃ + 0.776 𝐞₂₄ + -0.344 𝐞₃₄ + -2.78e-16 𝐞₁₂₃₄)

In [182]:
#8.17b hyperbolic rotor (boost)
S = random_simple_rotor(alg, signature=1)
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(1.29 + -0.5 𝐞₁₂ + -0.262 𝐞₁₃ + -0.0832 𝐞₁₄ + 1.59 𝐞₂₃ + 1.75 𝐞₂₄ + 0.654 𝐞₃₄ + 5.55e-17 𝐞₁₂₃₄,
 1.29 + -0.5 𝐞₁₂ + -0.262 𝐞₁₃ + -0.0832 𝐞₁₄ + 1.59 𝐞₂₃ + 1.75 𝐞₂₄ + 0.654 𝐞₃₄,
 1.07 + -0.234 𝐞₁₂ + -0.122 𝐞₁₃ + -0.0389 𝐞₁₄ + 0.741 𝐞₂₃ + 0.819 𝐞₂₄ + 0.305 𝐞₃₄,
 1.07 + -0.234 𝐞₁₂ + -0.122 𝐞₁₃ + -0.0389 𝐞₁₄ + 0.741 𝐞₂₃ + 0.819 𝐞₂₄ + 0.305 𝐞₃₄,
 1.07 + -0.234 𝐞₁₂ + -0.122 𝐞₁₃ + -0.0389 𝐞₁₄ + 0.741 𝐞₂₃ + 0.819 𝐞₂₄ + 0.305 𝐞₃₄ + 2.59e-17 𝐞₁₂₃₄,
 1.07 + -0.234 𝐞₁₂ + -0.122 𝐞₁₃ + -0.0389 𝐞₁₄ + 0.741 𝐞₂₃ + 0.819 𝐞₂₄ + 0.305 𝐞₃₄ + -3.73e-14 𝐞₁₂₃₄)

In [183]:
#8.17c 8.20 parabolic rotor (translation)
S = (e2 + e3 - e4) * e2
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(1 + -2 𝐞₂₃ + 2 𝐞₂₄,
 1 + -2 𝐞₂₃ + 2 𝐞₂₄,
 1 + -1 𝐞₂₃ + 1 𝐞₂₄,
 1 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄,
 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄,
 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄)

In [184]:
#8.21
B = S.grade(2) / (1 + S.grade(0))
B, (1+B)**2/(normsq(1+B)), S

(-0.5 𝐞₂₃ + 0.5 𝐞₂₄, 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄, 1 + -1 𝐞₂₃ + 1 𝐞₂₄)

In [185]:
#8.25
R, u = random_rotor(alg), random_r_blade(1, alg)
v = R.sw(u)
S = simple_rotor_sqrt(v * u / normsq(u))
U = S.reverse() * R
R, S * U, S * u, u * S.reverse(), U * u, u * U

(-1.81 + 3.37 𝐞₁₂ + 3.38 𝐞₁₃ + 3.49 𝐞₁₄ + -0.097 𝐞₂₃ + -2.73 𝐞₂₄ + -1.96 𝐞₃₄ + -1.27 𝐞₁₂₃₄,
 1.81 + -3.37 𝐞₁₂ + -3.38 𝐞₁₃ + -3.49 𝐞₁₄ + 0.097 𝐞₂₃ + 2.73 𝐞₂₄ + 1.96 𝐞₃₄ + 1.27 𝐞₁₂₃₄,
 -3.72 𝐞₁ + -0.365 𝐞₂ + -2.07 𝐞₃ + -4.31 𝐞₄ + -8.33e-17 𝐞₁₂₃ + -1.11e-16 𝐞₁₃₄ + 2.22e-16 𝐞₂₃₄,
 -3.72 𝐞₁ + -0.365 𝐞₂ + -2.07 𝐞₃ + -4.31 𝐞₄ + 1.11e-16 𝐞₁₃₄ + -1.11e-16 𝐞₂₃₄,
 -0.0995 𝐞₁ + -0.282 𝐞₂ + -0.334 𝐞₃ + -0.108 𝐞₄ + 0.09 𝐞₁₂₃ + -0.493 𝐞₁₂₄ + -0.55 𝐞₁₃₄ + 0.193 𝐞₂₃₄,
 -0.0995 𝐞₁ + -0.282 𝐞₂ + -0.334 𝐞₃ + -0.108 𝐞₄ + 0.09 𝐞₁₂₃ + -0.493 𝐞₁₂₄ + -0.55 𝐞₁₃₄ + 0.193 𝐞₂₃₄)

In [200]:
def rotor_split(R: MultiVector, alg: Algebra, tol=1e-6):
    # FIXME: when the tangent not well defined
    assert normsq(R.grade(2)) > tol, "zero bivector"
    tangent = R.grade(2) / R[0]
    tangents = bivector_split(tangent, alg)
    splits = sorted([normalize(1 + t) for t in tangents], key=lambda item: abs(item[0]))
    sign = np.round(gprod(r[0] for r in splits)/R[0])
    splits[0] = splits[0] * sign # to keep the sign of R
    return splits
        

R_splits = rotor_split(R, alg)
R, gprod(R_splits), gprod([blade_exp(simple_rotor_log(r)) for r in R_splits])

(-1.81 + 3.37 𝐞₁₂ + 3.38 𝐞₁₃ + 3.49 𝐞₁₄ + -0.097 𝐞₂₃ + -2.73 𝐞₂₄ + -1.96 𝐞₃₄ + -1.27 𝐞₁₂₃₄,
 -1.81 + 3.37 𝐞₁₂ + 3.38 𝐞₁₃ + 3.49 𝐞₁₄ + -0.097 𝐞₂₃ + -2.73 𝐞₂₄ + -1.96 𝐞₃₄ + -1.27 𝐞₁₂₃₄,
 -1.81 + 3.37 𝐞₁₂ + 3.38 𝐞₁₃ + 3.49 𝐞₁₄ + -0.097 𝐞₂₃ + -2.73 𝐞₂₄ + -1.96 𝐞₃₄ + -1.27 𝐞₁₂₃₄)

In [195]:
tangent = R.grade(2) / R[0]
tangents = bivector_split(tangent, alg)
splits = [normalize(1 + t) for t in tangents]
sign = np.round(gprod(r[0] for r in splits)/R[0])

In [196]:
tangents

[-0.258 𝐞₁₂ + -1.54 𝐞₁₃ + -0.929 𝐞₂₃ + -0.846 𝐞₁₄ + -0.279 𝐞₂₄ + 1.38 𝐞₃₄,
 -1.6 𝐞₁₂ + -0.324 𝐞₁₃ + 0.982 𝐞₂₃ + -1.08 𝐞₁₄ + 1.79 𝐞₂₄ + -0.3 𝐞₃₄]

In [197]:
splits

[0.79 + -0.204 𝐞₁₂ + -1.22 𝐞₁₃ + -0.669 𝐞₁₄ + -0.734 𝐞₂₃ + -0.221 𝐞₂₄ + 1.09 𝐞₃₄,
 2.3 + -3.68 𝐞₁₂ + -0.745 𝐞₁₃ + -2.48 𝐞₁₄ + 2.26 𝐞₂₃ + 4.1 𝐞₂₄ + -0.688 𝐞₃₄]

In [199]:
def inv(A: MultiVector, tol=1e-6):
    Ar = A.reverse()
    n = A.sp(Ar)[0]
    assert abs(n) > tol, f"norm {n}"
    return Ar / n

R * inv(splits[0]), R * inv(splits[1])

(-2.3 + 3.68 𝐞₁₂ + 0.745 𝐞₁₃ + 2.48 𝐞₁₄ + -2.26 𝐞₂₃ + -4.1 𝐞₂₄ + 0.688 𝐞₃₄ + -2.22e-16 𝐞₁₂₃₄,
 -0.79 + 0.204 𝐞₁₂ + 1.22 𝐞₁₃ + 0.669 𝐞₁₄ + 0.734 𝐞₂₃ + 0.221 𝐞₂₄ + -1.09 𝐞₃₄ + -1.78e-15 𝐞₁₂₃₄)

In [198]:
sign

-1.0

In [193]:
[blade_exp(simple_rotor_log(r)) for r in R_splits]

/var/folders/4w/twr20pmj5p356r2dcqzxg9v00000gn/T/ipykernel_97258/1347492118.py:5: RuntimeWarning: invalid value encountered in arccosh
  return np.arccosh(R[0]) / norm(blade) * blade


[0.79 + -0.204 𝐞₁₂ + -1.22 𝐞₁₃ + -0.669 𝐞₁₄ + -0.734 𝐞₂₃ + -0.221 𝐞₂₄ + 1.09 𝐞₃₄,
 None]

In [53]:
#8.10 a spinor is an even versor for odd dimension (a Dirac representation);
# a sum of an even versor and its dual for even dimension (two Weyl representations)
def random_study_number(alg):
    a, b = np.random.rand(2)
    return (a + b*alg.pseudoscalar((1,)))

def random_spinor(alg: Algebra):
    if alg.d % 4:
        return random_even_versor(alg)
    return random_study_number(alg) * random_even_versor(alg)

random_spinor(Algebra(3)), random_spinor(Algebra(4))


(0.343 + 0.0121 𝐞₁₂ + 0.0756 𝐞₁₃ + 0.0557 𝐞₂₃,
 0.409 + 0.115 𝐞₁₂ + 0.327 𝐞₁₃ + 0.453 𝐞₁₄ + -0.476 𝐞₂₃ + 0.321 𝐞₂₄ + -0.11 𝐞₃₄ + 0.383 𝐞₁₂₃₄)

In [54]:
# A pointor depends on a point
def random_point(alg: Algebra):
    return random_r_blade(alg.d - 1, alg)

O = random_point(alg)

# point reflection on the point itself is a pointor
terms_ratio(O, O.sw(O))

array([-13.40003444, -13.40003444, -13.40003444, -13.40003444])

In [55]:
# versors not crossing O-subspace are pointers 
vectors = blade_split(O, alg)
B1, B2 = wedge(vectors[:2]), wedge(vectors[1:])
Od = O.dual()
terms_ratio(O, Od.sw(O)), terms_ratio(B1.sw(O), O), terms_ratio(vectors[0].sw(O), O), 

(array([-13.40003444, -13.40003444, -13.40003444, -13.40003444]),
 array([7.53618382, 7.53618382, 7.53618382, 7.53618382]),
 array([7.69296609, 7.69296609, 7.69296609, 7.69296609]))

In [56]:
# rotors with reflections are pointers
R1 = blade_exp(3.3*B1)
R2 = blade_exp(4.4*B2)
terms_ratio((Od * R1 * R2).sw(O), O), terms_ratio((Od * vectors[2] * R1).sw(O), O)

(array([-0.07462667, -0.07462667, -0.07462667, -0.07462667]),
 array([0.07462854, 0.07462854, 0.07462854, 0.07462854]))

In [57]:
# rotors cross O-subspace may not be pointers
terms_ratio(blade_exp(Od ^ vectors[2]).sw(O), O)

array([116.06883467,   1.15587383,   1.15587383,   1.15587383])

In [109]:
# A general pointor suggested by "From Invariant Decomposition to Spinors"?
# How do we find the point specified by a pointor?
def random_odd_versor(alg):
    d = alg.d
    if not (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


# Actually I'm not sure these generate pointors
# def random_pointor(alg: Algebra):
#     return random_odd_versor(alg) * random_even_versor(alg)
    
# def random_pointor(alg: Algebra):
#     v = random_r_blade(1, alg)
#     return (np.random.rand() + v) * random_even_versor(alg)

# FIXME: check degree of freedom
def random_pointor(alg: Algebra):
    v = random_r_blade(1, alg)
    return (np.random.rand() + v) * random_versor(2, alg)

pointor = random_pointor(alg)
pointor

0.0118 + 0.136 𝐞₁ + 0.412 𝐞₂ + 0.385 𝐞₃ + 0.502 𝐞₄ + -0.0264 𝐞₁₂ + -0.031 𝐞₁₃ + -0.00605 𝐞₁₄ + -0.0271 𝐞₂₃ + 0.0781 𝐞₂₄ + 0.0977 𝐞₃₄ + -0.092 𝐞₁₂₃ + -0.0108 𝐞₁₂₄ + 0.00839 𝐞₁₃₄ + -0.283 𝐞₂₃₄

In [110]:
# the point-grade
O = pointor.grade(3)
terms_ratio(pointor.sw(O), O)

array([0.00031733, 0.00031733, 0.00031733, 0.00031733])

### Algebraic Representation of Linear Multivector Functions
effect of F on the structure of G -> express F in terms of algebraic operations on multivectors


In [89]:
#9.8
def protraction(F, Y):
    return curl(lambda x: F(x ^ Y), 0, alg, grade=1)

def contraction(F, Y):
    return div(lambda x: F(x ^ Y), 0, alg, grade=1)

def traction(F, Y):
    return derivative(lambda x: F(x ^ Y), 0, alg, grade=1)

A, B = [random_multivector(alg) for _ in range(2)]
F = lambda X: A * X.grade(3)
x, Y = random_r_blade(1, alg), random_r_blade(2, alg)

traction(F, Y), contraction(F, Y) + protraction(F, Y)

(-0.995 + 0.485 𝐞₁ + -1.14 𝐞₂ + 1.27 𝐞₃ + 1.36 𝐞₄ + 0.518 𝐞₁₂ + -0.747 𝐞₁₃ + -0.796 𝐞₁₄ + -0.857 𝐞₂₃ + -0.868 𝐞₂₄ + 0.228 𝐞₃₄ + -0.0144 𝐞₁₂₃ + 0.0887 𝐞₁₂₄ + -1.15 𝐞₁₃₄ + 0.172 𝐞₂₃₄ + -0.424 𝐞₁₂₃₄,
 -0.995 + 0.485 𝐞₁ + -1.14 𝐞₂ + 1.27 𝐞₃ + 1.36 𝐞₄ + 0.518 𝐞₁₂ + -0.747 𝐞₁₃ + -0.796 𝐞₁₄ + -0.857 𝐞₂₃ + -0.868 𝐞₂₄ + 0.228 𝐞₃₄ + -0.0144 𝐞₁₂₃ + 0.0887 𝐞₁₂₄ + -1.15 𝐞₁₃₄ + 0.172 𝐞₂₃₄ + -0.424 𝐞₁₂₃₄)

In [92]:
#9.9 symmetric multiform not necessarily protractionless
B2 = B.grade(2)
X = random_r_blade(2, alg)
G0 = lambda x: x|B2  # skew symmetric 1 form
G = lambda X: outermorphism(G0, X, alg)  # symmetric multiform
G(X) | Y, G(Y) | X, protraction(G, x), contraction(G0, x)

(0.0661, 0.0661, 1.38 𝐞₁₂₃ + 1.19 𝐞₁₂₄ + -0.298 𝐞₁₃₄ + 0.808 𝐞₂₃₄, )

In [93]:
# if the bivector is simple instead
G1 = lambda X: outermorphism(lambda x: x|Y, X, alg)
protraction(G1, x)

-6.94e-17 𝐞₁₂₃ + -7.63e-17 𝐞₁₂₄ + 1.73e-17 𝐞₁₃₄ + -5.55e-17 𝐞₂₃₄

In [97]:
#9.11 simplicial curl is the max grade of simplicial derivative
simplicial_derivative(lambda vectors: G(wedge(vectors)), np.zeros(2), alg).grade(4)

3.5 𝐞₁₂₃₄

In [99]:
#9.12b
simplicial_derivative(lambda vectors: G1(wedge(vectors)), np.zeros(2), alg)

0.0983 + -1.82e-17 𝐞₁₃ + -1.3e-18 𝐞₁₄ + -1.94e-16 𝐞₁₂₃₄

In [130]:
#9.13 contractionless
trace = simplicial_derivative(lambda vectors: G(wedge(vectors)), np.zeros(2), alg)[0]
G2 = lambda X: outermorphism(G0, X, alg) - trace/4 * X  # symmetric multiform
simplicial_derivative(lambda vectors: G2(wedge(vectors)), np.zeros(2), alg)

0.1 + 2.78e-17 𝐞₁₃ + 3.5 𝐞₁₂₃₄

In [135]:
#9.13 zero simplicial divergence not necessarily contractionless
trace = simplicial_derivative(lambda vectors: G(wedge(vectors)), np.zeros(2), alg)[0]
G2 = lambda X: outermorphism(G0, X, alg) - trace/6 * X  # symmetric multiform
protraction(G2, x), contraction(G2, x), simplicial_derivative(lambda vectors: G2(wedge(vectors)), np.zeros(2), alg)

(1.38 𝐞₁₂₃ + 1.19 𝐞₁₂₄ + -0.298 𝐞₁₃₄ + 0.808 𝐞₂₃₄,
 0.0556 𝐞₁ + 0.72 𝐞₂ + -0.815 𝐞₃ + -1.06 𝐞₄,
 -5.55e-17 + 2.78e-17 𝐞₁₃ + 3.5 𝐞₁₂₃₄)

In [122]:
trace

0.0999653782692127

In [ ]:
B3 = B.grade(3)
X = random_r_blade(2, alg)
G0 = lambda x: x|B2  # skew symmetric 1 form
G = lambda X: outermorphism(G0, X, alg)  # symmetric multiform
G(X) | Y, G(Y) | X, protraction(G, x), contraction(G0, x)